# Chapter 17 - Decision Trees

In [1]:
import math
import random
from collections import Counter
from collections import defaultdict
from functools import partial

import numpy as np
import matplotlib.pyplot as plt

Decision trees can easily handle a mix of numer and categorical attributes and can even classify data for which attributes are missing. Easy to understand and interpret, and the process of reaching a prediction is transparent.

However, finding the optimal decision tree for a training set is a computationally hard problem. It's easy to build a decision tree that is overfitted to the training data, which generalize poorly to unseen data.

Decision trees can be divided into classification trees and regression trees.

Entropy: uncertainty associated with the data. Imagine a set $S$ of data, each member of which is labeled and belongs to a set of finite classes $C_1, ..., C_n$. If all data belongs to a single class, there is no uncertainty and low entropy. If the data is spread out evenly amongst all classes, then there is high entropy. In math terms, $p_i$ is the proportion of data labeld as class $c_i$. Then etropy is: <br>

<center> $H(S) = - p_1 log_2 p_1 - ... - p_n log_2 p_n$, where 0 log 0 = 0. </center>
$-p_i log_2 p_i$ is non-negative and close to zero precisely when p_i is either close to zero or close to one. So entropy is small when every $p_i$ is close to 0 or 1, and large when $p_i$'s are not close to 1 or 0.

In [2]:
def entropy(class_probabilities):
    """given a list of class probabilities, compute the entropy"""
    return sum(-p * math.log(p,2) for p in class_probabilities if p)

In [3]:
def class_probabilites(labels):
    total_count = len(labels)
    return [count / total_count for count in Counter(labels).values()]

In [4]:
def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilites(labels)
    return entropy(probabilities)

In [5]:
def partition_entropy(subsets):
    """find the entropy from this partition of data into subsets
    sub sets is a list of lists of labeled data"""
    
    total_count = sum(len(subset) for subset in subsets)
    
    return sum( data_entropy(subset) * len(subset) / total_count for subset in subsets )

In [6]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
]

In [7]:
def partition_by(inputs, attribute):
    """each input is a pair (attribute_dict, label)
    returns a dict : attribute_value -> inputs"""
    
    groups = defaultdict(list)
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)
    return groups    

In [8]:
def partition_entropy_by(inputs, attribute):
    """computes the entropy corresponding to the give partition"""
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [9]:
for key in ['level', 'lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8601317128547441
tweets 0.7884504573082896
phd 0.8921589282623617


In [10]:
senior_inputs = [(input, label) for input, label in inputs if input["level"] == "Senior"]


In [11]:
for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(senior_inputs, key))

lang 0.4
tweets 0.0
phd 0.9509775004326938


In [12]:
mid_inputs = [(input, label) for input, label in inputs if input["level"] == "Mid"]

junior_inputs = [(input, label) for input, label in inputs if input["level"] == "Junior"]

In [13]:
for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(mid_inputs, key))

lang 0.0
tweets 0.0
phd 0.0


In [14]:
for key in ['lang', 'tweets', 'phd']:
    print(key, partition_entropy_by(junior_inputs, key))

lang 0.9509775004326938
tweets 0.9509775004326938
phd 0.0


In [15]:
junior_phd_inputs = [(input, label) for input, label in junior_inputs if input["phd"] == 'yes']

In [16]:
for key in ['lang', 'tweets']:
    print(key, partition_entropy_by(junior_phd_inputs, key))

lang 0.0
tweets 0.0


## Putting it all together

We define a tree to be one of the following:
* True
* False
* a tuple (attribute, subtree_dict)

Here True represents a leaf node that returns True for any input False represents a leaf node that returns False for any input, and a tuple represents a decision node that, for any input, finds its attribute value, and classifies the input using the corresponding subtree.

In [17]:
def classify(tree, input):
    """classify the input using the given decision tree"""
    
    #if this is a leaf node, return its value
    if tree in [True, False]:
        return tree
    
    #otherwise this tree consists of an attribute to split on and a dictionary whose keys are values of that atrribute
    # and whose values of are subtrees to consider next attribute, subtree_dict = tree
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)   #None if input is missing
    
    if subtree_key not in subtree_dict:   # if no subtree key,
        subtree_key = None                # we'll use the None subtree
    
    subtree = subtree_dict[subtree_key]
    return classify(subtree, input)
        

In [18]:
def build_tree_id3(inputs, split_candidates=None):
    
    #if this is our first pass, all keys of the first input are split candidates
    
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
        
    # count Trues and Falses in the inputs
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0: return False # no trues, return "False" leaf
    if num_falses == 0: return True # no fales, return "True" leaf
    
    if not split_candidates:
        return num_trues >= num_falses
    
    #otherwise split on the best attribute
    best_attribute = min(split_candidates, key=partial(partition_entropy_by, inputs))
    
    partitions = partition_by(inputs, best_attribute)
    new_candidates = [a for a in split_candidates if a != best_attribute]
    
    # recursively build the subtrees
    subtrees = {attribute_value : build_tree_id3(subset, new_candidates) 
                for attribute_value, subset in partitions.items() }
    
    subtrees[None] = num_trues > num_falses   # default case
    
    return (best_attribute, subtrees)
    

In [19]:
tree = build_tree_id3(inputs)

In [20]:
print("Junior / Java / tweets / no phd", classify(tree, 
        { "level" : "Junior", 
          "lang" : "Java", 
          "tweets" : "yes", 
          "phd" : "no"} ) )

Junior / Java / tweets / no phd True


In [21]:
print("Junior / Java / tweets / phd", classify(tree, 
        { "level" : "Junior", 
                 "lang" : "Java", 
                 "tweets" : "yes", 
                 "phd" : "yes"} ) )

    

Junior / Java / tweets / phd False


In [22]:
print("Intern", classify(tree, { "level" : "Intern" } ))


Intern True


In [23]:
print("Senior", classify(tree, { "level" : "Senior" } ))

Senior False


In [24]:
def forest_classify(trees, input):
    votes = [classify(tree, input) for tree in trees]
    vote_counts = Counter(votes)
    return vote_counts.most_common(1)[0][0]

Introduce randomness by fitting trees to random subsets of the data (bootstrapping) and then test the trees on the remaining data points. Choose the tree in the forest with the best fit on the test sets.

You can also introduce randomness by changing the way we chose the best_attribute to split on. Rather than looking at all the remaining attributes, we first choose a random subset of them and then split on whichever of those is best:

In [25]:
# example function:

def random_attribute_selection():
    
    if len(split_candidates) <= self.num_split_candidates:
        sampled_split_candidates = split_candidates
        
    else:
        sampled_split_candidates = random.smaple(split_candidates, self.num_split_candidates)
        
    best_attribute = min(sampled_split_candidates, key = partial(partition_entropy_by, inputs))
    
    partitions = partitions_by(inputs, best_attribute)